<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#!pip install polars

In [1]:
import polars as pl
import random
import string


# Generacion de Datos

In [3]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""skhji""",15,55.760864,"2014-01-13 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1954
2,"""yjcbu""",3,44.590108,"2020-08-08 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4268
3,"""bsmfy""",2,33.875485,"2020-06-01 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3543
4,"""apwgf""",20,65.586471,"2017-05-02 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4595
5,"""kecml""",10,10.024797,"2011-10-16 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2694


In [5]:
df = df.lazy()
df

<LazyFrame [6 cols, {"id_venta": Int64 … "cliente_id": Int64}] at 0x7FCCDC3B6690>

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [18]:

lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación

resultado = (
    lazy_df
    .filter(pl.col('cantidad') < 5)
    .with_columns((pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta"))
    .drop("precio_unitario")
    .group_by('producto')
    .agg([
        pl.sum("total_venta").alias("total_vendido"), 
        pl.sum("cantidad").alias("cantidad_total_vendida") 
    ])
    .sort("total_vendido", descending=True)
)

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)


shape: (193, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ tfawk    ┆ 398.142873    ┆ 4                      │
│ dqqig    ┆ 389.889814    ┆ 4                      │
│ zcoez    ┆ 388.229993    ┆ 4                      │
│ ngtqf    ┆ 378.353501    ┆ 4                      │
│ gqnon    ┆ 378.081705    ┆ 4                      │
│ …        ┆ …             ┆ …                      │
│ abqqo    ┆ 19.846645     ┆ 1                      │
│ pvawm    ┆ 17.319557     ┆ 1                      │
│ wgesw    ┆ 13.97716      ┆ 1                      │
│ inysi    ┆ 11.557242     ┆ 1                      │
│ aawvp    ┆ 10.006594     ┆ 1                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [23]:
with open("resultados_df.parquet", "wb") as f: 
    resultado_df.write_parquet(f)

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [25]:
with open("resultados_df.parquet", "rb") as f: 
    df_parquet = pl.read_parquet(f)
    
df_parquet

producto,total_vendido,cantidad_total_vendida
str,f64,i64
"""tfawk""",398.142873,4
"""dqqig""",389.889814,4
"""zcoez""",388.229993,4
"""ngtqf""",378.353501,4
"""gqnon""",378.081705,4
…,…,…
"""abqqo""",19.846645,1
"""pvawm""",17.319557,1
"""wgesw""",13.97716,1
